In [2]:
import pandas as pd

# Read the CSV file
df = pd.read_csv(r'C:\Users\Andrei.Baidurov\AutoTemplate\Data\Inputdata.csv')

print(f"Dataset loaded successfully!")
print(f"Shape: {df.shape}")
print(f"\nColumn names:")
print(df.columns.tolist())
print(f"\nData types:")
print(df.dtypes)
print(f"\nBasic statistics:")
print(df.describe())
print(f"\nFirst 5 rows:")
print(df.head())
print(f"\nMissing values:")
print(df.isnull().sum())

Dataset loaded successfully!
Shape: (34805, 4)

Column names:
['Cost', 'City', 'Country', 'Hotel_ID']

Data types:
Cost        float64
City         object
Country      object
Hotel_ID    float64
dtype: object

Basic statistics:
                Cost      Hotel_ID
count   34805.000000  3.347200e+04
mean      640.166749  1.460856e+07
std      1352.096640  2.554255e+07
min     -2008.200000  1.000144e+06
25%       137.400000  1.382663e+06
50%       296.000000  2.150777e+06
75%       668.180000  7.650056e+06
max    118650.000000  9.997236e+07

First 5 rows:
     Cost                 City        Country    Hotel_ID
0  358.65                Milan          Italy   3372898.0
1  178.56                Krabi       Thailand   1847182.0
2   68.48               Madrid          Spain  25506072.0
3  563.27  Fort Lauderdale, FL  United States   2321095.0
4   17.15                  NaN          Egypt         NaN

Missing values:
Cost           0
City        1333
Country        0
Hotel_ID    1333
dtype: in

In [3]:
# Drop rows where Hotel_ID is empty (NaN)
df = df.dropna(subset=['Hotel_ID'])

print(f"Dataset after dropping empty Hotel_ID:")
print(f"Shape: {df.shape}")
print(f"\nMissing values:")
print(df.isnull().sum())

Dataset after dropping empty Hotel_ID:
Shape: (33472, 4)

Missing values:
Cost        0
City        0
Country     0
Hotel_ID    0
dtype: int64


In [4]:
# Calculate quantity of reservations and sum of cost by Hotel_ID
hotel_stats = df.groupby('Hotel_ID').agg({
    'Cost': ['count', 'sum'],
    'City': 'first',
    'Country': 'first'
}).reset_index()

# Flatten column names
hotel_stats.columns = ['Hotel_ID', 'Quantity_Reservations', 'Total_Cost', 'City', 'Country']

print(f"Hotel statistics:")
print(f"Shape: {hotel_stats.shape}")
print(f"\nFirst 5 rows:")
print(hotel_stats.head())
print(f"\nBasic statistics for reservations:")
print(hotel_stats['Quantity_Reservations'].describe())

Hotel statistics:
Shape: (13206, 5)

First 5 rows:
    Hotel_ID  Quantity_Reservations  Total_Cost                  City  \
0  1000144.0                      1      154.28  Oakbrook Terrace, IL   
1  1000354.0                      1      699.07            Dusseldorf   
2  1000479.0                      8     1273.89         Arlington, VA   
3  1000523.0                      1      178.65               Antalya   
4  1000538.0                      4     1037.09             Marseille   

         Country  
0  United States  
1        Germany  
2  United States  
3         Turkey  
4         France  

Basic statistics for reservations:
count    13206.000000
mean         2.534605
std          4.401539
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max        104.000000
Name: Quantity_Reservations, dtype: float64


In [5]:
# Convert Hotel_ID to integer to remove .0
hotel_stats['Hotel_ID'] = hotel_stats['Hotel_ID'].astype(int)

print("Hotel_ID converted to integer:")
print(hotel_stats.head())
print(f"\nData types after conversion:")
print(hotel_stats.dtypes)

Hotel_ID converted to integer:
   Hotel_ID  Quantity_Reservations  Total_Cost                  City  \
0   1000144                      1      154.28  Oakbrook Terrace, IL   
1   1000354                      1      699.07            Dusseldorf   
2   1000479                      8     1273.89         Arlington, VA   
3   1000523                      1      178.65               Antalya   
4   1000538                      4     1037.09             Marseille   

         Country  
0  United States  
1        Germany  
2  United States  
3         Turkey  
4         France  

Data types after conversion:
Hotel_ID                   int64
Quantity_Reservations      int64
Total_Cost               float64
City                      object
Country                   object
dtype: object


In [7]:
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Add Search column with values between 1000 and 50000
hotel_stats['Search'] = np.random.randint(1000, 50001, size=len(hotel_stats))

# Add Result column - should be <= Search and realistic
# Using a probability distribution that makes sense:
# - Higher search volumes tend to have higher response rates
# - But with some randomness to make it realistic
hotel_stats['Result'] = hotel_stats['Search'].apply(lambda x: 
    np.random.randint(
        max(1, int(x * 0.1)),  # Minimum 10% response rate
        int(x * np.random.uniform(0.3, 0.8)) + 1  # 30-80% response rate with randomness
    )
)

# Ensure Result is never greater than Search
hotel_stats['Result'] = np.minimum(hotel_stats['Result'], hotel_stats['Search'])

print("Added Search and Result columns:")
print(hotel_stats[['Hotel_ID', 'Search', 'Result']].head(10))
print(f"\nSearch column statistics:")
print(hotel_stats['Search'].describe())
print(f"\nResult column statistics:")
print(hotel_stats['Result'].describe())
print(f"\nResult/Search ratio statistics:")
print((hotel_stats['Result'] / hotel_stats['Search']).describe())

Added Search and Result columns:
   Hotel_ID  Search  Result
0   1000144   16795    5634
1   1000354    1860     590
2   1000479   39158   21919
3   1000523   45732   12653
4   1000538   12284    7850
5   1000600    7265    1291
6   1000863   17850    2488
7   1000888   38194   23071
8   1001193   22962    6623
9   1001269   48191   12433

Search column statistics:
count    13206.000000
mean     25394.198395
std      14114.139362
min       1002.000000
25%      13175.500000
50%      25259.500000
75%      37634.750000
max      49998.000000
Name: Search, dtype: float64

Result column statistics:
count    13206.000000
mean      8301.105028
std       6467.926505
min        131.000000
25%       3351.750000
50%       6643.000000
75%      11731.000000
max      38311.000000
Name: Result, dtype: float64

Result/Search ratio statistics:
count    13206.000000
mean         0.325535
std          0.153346
min          0.099411
25%          0.200671
50%          0.300633
75%          0.427206
max     

In [8]:
print("Current hotel_stats table:")
print(f"Shape: {hotel_stats.shape}")
print(f"\nColumn names: {hotel_stats.columns.tolist()}")
print(f"\nFirst 10 rows:")
print(hotel_stats.head(10))
print(f"\nLast 5 rows:")
print(hotel_stats.tail())
print(f"\nData types:")
print(hotel_stats.dtypes)

Current hotel_stats table:
Shape: (13206, 7)

Column names: ['Hotel_ID', 'Quantity_Reservations', 'Total_Cost', 'City', 'Country', 'Search', 'Result']

First 10 rows:
   Hotel_ID  Quantity_Reservations  Total_Cost                  City  \
0   1000144                      1      154.28  Oakbrook Terrace, IL   
1   1000354                      1      699.07            Dusseldorf   
2   1000479                      8     1273.89         Arlington, VA   
3   1000523                      1      178.65               Antalya   
4   1000538                      4     1037.09             Marseille   
5   1000600                      8     2218.61                Madrid   
6   1000863                      2     1178.82              Mallorca   
7   1000888                      4     4277.37              Istanbul   
8   1001193                      1     1597.82            Interlaken   
9   1001269                      2     8561.52                Rhodes   

         Country  Search  Result  
0  Un

In [9]:
# Save hotel_stats to a new CSV file in the same folder
output_path = r'C:\Users\Andrei.Baidurov\AutoTemplate\Data\hotel_statistics_clean.csv'
hotel_stats.to_csv(output_path, index=False)

print(f"Hotel statistics saved successfully to: {output_path}")
print(f"File contains {len(hotel_stats)} rows and {len(hotel_stats.columns)} columns")

Hotel statistics saved successfully to: C:\Users\Andrei.Baidurov\AutoTemplate\Data\hotel_statistics_clean.csv
File contains 13206 rows and 7 columns


In [10]:
# Truncate Cost values to 2 decimal places
hotel_stats['Total_Cost'] = hotel_stats['Total_Cost'].round(2)

print("Hotel statistics with Total_Cost rounded to 2 decimal places:")
print(hotel_stats[['Hotel_ID', 'Total_Cost']].head(10))
print(f"\nTotal_Cost column statistics:")
print(hotel_stats['Total_Cost'].describe())

# Save the updated hotel_stats to a new CSV file
updated_output_path = r'C:\Users\Andrei.Baidurov\AutoTemplate\Data\hotel_statistics_rounded.csv'
hotel_stats.to_csv(updated_output_path, index=False)

print(f"\nUpdated hotel statistics saved to: {updated_output_path}")
print(f"File contains {len(hotel_stats)} rows and {len(hotel_stats.columns)} columns")

Hotel statistics with Total_Cost rounded to 2 decimal places:
   Hotel_ID  Total_Cost
0   1000144      154.28
1   1000354      699.07
2   1000479     1273.89
3   1000523      178.65
4   1000538     1037.09
5   1000600     2218.61
6   1000863     1178.82
7   1000888     4277.37
8   1001193     1597.82
9   1001269     8561.52

Total_Cost column statistics:
count     13206.000000
mean       1679.604641
std        4049.829854
min       -2008.200000
25%         242.880000
50%         606.460000
75%        1602.887500
max      198619.650000
Name: Total_Cost, dtype: float64

Updated hotel statistics saved to: C:\Users\Andrei.Baidurov\AutoTemplate\Data\hotel_statistics_rounded.csv
File contains 13206 rows and 7 columns


In [ ]:
%pip install pyodbc

In [9]:
import pandas as pd

In [10]:
# Read the CSV file using the existing csv_path variable
hotel_data_rounded = pd.read_csv(csv_path)

print(f"File loaded successfully!")
print(f"Shape: {hotel_data_rounded.shape}")
print(f"Columns: {hotel_data_rounded.columns.tolist()}")
print(f"\nFirst 5 rows:")
print(hotel_data_rounded.head())
print(f"\nData types:")
print(hotel_data_rounded.dtypes)

File loaded successfully!
Shape: (13206, 7)
Columns: ['Hotel_ID', 'Quantity_Reservations', 'Total_Cost', 'City', 'Country', 'Search', 'Result']

First 5 rows:
   Hotel_ID  Quantity_Reservations  Total_Cost                  City  \
0   1000144                      1      154.28  Oakbrook Terrace, IL   
1   1000354                      1      699.07            Dusseldorf   
2   1000479                      8     1273.89         Arlington, VA   
3   1000523                      1      178.65               Antalya   
4   1000538                      4     1037.09             Marseille   

         Country  Search  Result  
0  United States   16795    5634  
1        Germany    1860     590  
2  United States   39158   21919  
3         Turkey   45732   12653  
4         France   12284    7850  

Data types:
Hotel_ID                   int64
Quantity_Reservations      int64
Total_Cost               float64
City                      object
Country                   object
Search              

In [13]:
import pyodbc

try:
    # Create connection to SQL Server database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    # Clear existing data in the table (optional)
    cursor.execute("DELETE FROM [dbo].[InputApidata]")
    print("Existing data cleared from [dbo].[InputApidata]")
    
    # Insert data row by row
    insert_query = """
    INSERT INTO [dbo].[InputApidata] 
    (Hotel_ID, Quantity_Reservations, Total_Cost, City, Country, Search, Result)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    
    # Convert DataFrame to list of tuples for insertion
    data_to_insert = hotel_data_rounded.values.tolist()
    
    # Execute batch insert
    cursor.executemany(insert_query, data_to_insert)
    
    # Commit the transaction
    conn.commit()
    
    print(f"Successfully inserted {len(hotel_data_rounded)} rows into [dbo].[InputApidata]")
    
    # Verify the insertion
    cursor.execute("SELECT COUNT(*) FROM [dbo].[InputApidata]")
    count = cursor.fetchone()[0]
    print(f"Total rows in table after insertion: {count}")
    
except Exception as e:
    print(f"Error occurred: {str(e)}")
    if 'conn' in locals():
        conn.rollback()
        
finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
    print("Database connection closed.")

Error occurred: ('IM002', '[IM002] [Microsoft][Administrador de controladores ODBC] No se encuentra el nombre del origen de datos y no se especificó ningún controlador predeterminado (0) (SQLDriverConnect)')
Database connection closed.


In [8]:
import pandas as pd

In [22]:
import pyodbc

# First, let's check what ODBC drivers are available
print("Available ODBC drivers:")
drivers = [x for x in pyodbc.drivers() if 'SQL Server' in x]
for driver in drivers:
    print(f"  - {driver}")

if not drivers:
    print("No SQL Server ODBC drivers found!")
    print("Please install Microsoft ODBC Driver for SQL Server")
    raise Exception("No SQL Server ODBC drivers available")

# Use the first available driver
available_driver = drivers[0]
print(f"\nUsing driver: {available_driver}")

# Define connection strings to try in order using the available driver
connection_strings = [
    # Option 1: SQL Server Authentication with username and password
    f'DRIVER={{{available_driver}}};SERVER=upgradeserverdb-akb.database.windows.net;DATABASE=AutTemplate;UID=andreikb;PWD=Travellanda2026*!*;Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;',
    
    # Option 2: Alternative with different encryption settings
    f'DRIVER={{{available_driver}}};SERVER=upgradeserverdb-akb.database.windows.net;DATABASE=AutTemplate;UID=andreikb;PWD=Travellanda2026*!*;Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=30;',
    
    # Option 3: Azure Active Directory Interactive (if available)
    f'DRIVER={{{available_driver}}};SERVER=upgradeserverdb-akb.database.windows.net;DATABASE=AutTemplate;Authentication=ActiveDirectoryInteractive;Connection Timeout=30;'
]

# Use existing hotel_data_rounded DataFrame (already loaded)
print(f"Data to insert: {len(hotel_data_rounded)} rows")

# Try each connection string until one works
conn = None
for i, conn_str in enumerate(connection_strings, 1):
    try:
        print(f"Attempting connection method {i}...")
        conn = pyodbc.connect(conn_str, timeout=30)
        print(f"Successfully connected using method {i}")
        break
    except Exception as e:
        print(f"Connection method {i} failed: {str(e)}")
        if i == len(connection_strings):
            print("All connection methods failed!")
            raise

if conn is None:
    raise Exception("Could not establish database connection")

try:
    cursor = conn.cursor()
    
    # Test connection with a simple query
    cursor.execute("SELECT 1")
    print("Database connection verified")
    
    # Clear existing data in the table (optional)
    cursor.execute("DELETE FROM [dbo].[InputApidata]")
    conn.commit()  # Commit the delete operation
    print("Existing data cleared from [dbo].[InputApidata]")
    
    # Insert data row by row
    insert_query = """
    INSERT INTO [dbo].[InputApidata] 
    (Hotel_ID, Quantity_Reservations, Total_Cost, City, Country, Search, Result)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    
    # Convert DataFrame to list of tuples for insertion
    data_to_insert = hotel_data_rounded.values.tolist()
    
    # Execute batch insert in chunks to avoid timeout
    chunk_size = 500  # Reduced chunk size for better reliability
    total_inserted = 0
    
    for i in range(0, len(data_to_insert), chunk_size):
        chunk = data_to_insert[i:i + chunk_size]
        cursor.executemany(insert_query, chunk)
        conn.commit()  # Commit each chunk
        total_inserted += len(chunk)
        print(f"Inserted {total_inserted}/{len(data_to_insert)} rows...")
    
    print(f"Successfully inserted {len(hotel_data_rounded)} rows into [dbo].[InputApidata]")
    
    # Verify the insertion
    cursor.execute("SELECT COUNT(*) FROM [dbo].[InputApidata]")
    count = cursor.fetchone()[0]
    print(f"Total rows in table after insertion: {count}")
    
except Exception as e:
    print(f"Error occurred during database operation: {str(e)}")
    if conn:
        conn.rollback()
    raise
        
finally:
    if 'cursor' in locals():
        cursor.close()
    if conn:
        conn.close()
    print("Database connection closed.")

Available ODBC drivers:
  - SQL Server

Using driver: SQL Server
Data to insert: 13206 rows
Attempting connection method 1...
Successfully connected using method 1
Database connection verified
Existing data cleared from [dbo].[InputApidata]
Inserted 500/13206 rows...
Inserted 1000/13206 rows...
Inserted 1500/13206 rows...
Inserted 2000/13206 rows...
Inserted 2500/13206 rows...
Inserted 3000/13206 rows...
Inserted 3500/13206 rows...
Inserted 4000/13206 rows...
Inserted 4500/13206 rows...
Inserted 5000/13206 rows...
Inserted 5500/13206 rows...
Inserted 6000/13206 rows...
Inserted 6500/13206 rows...
Inserted 7000/13206 rows...
Inserted 7500/13206 rows...
Inserted 8000/13206 rows...
Inserted 8500/13206 rows...
Inserted 9000/13206 rows...
Inserted 9500/13206 rows...
Inserted 10000/13206 rows...
Inserted 10500/13206 rows...
Inserted 11000/13206 rows...
Inserted 11500/13206 rows...
Inserted 12000/13206 rows...
Inserted 12500/13206 rows...
Inserted 13000/13206 rows...
Inserted 13206/13206 rows